In [1]:
from google.colab import drive ; drive.mount("/content/drive")

Mounted at /content/drive


In [59]:
# Install the Pre Requirments
!pip install transformers==3.4.0
!pip install pytypo
!pip install tensorflow==2.2.0 
!pip install Sastrawi
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 215kB 16.2MB/s 


In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import pytypo
from re import sub,compile,UNICODE
import time
from collections import Counter
import transformers
import pickle
import sastrawi
import tensorflow_addons as tfa
import nltk 
nltk.download("stopwords");nltk.download("punkt")
import tensorflow as tf
from googletrans import Translator
pd.set_option("max_rows",None)

print(tf.__version__)
print('Keras version: %s' % tf.keras.__version__)
print(f"Transformes {transformers.__version__}")
MAX_LENGTH = 45
AUTO = tf.data.experimental.AUTOTUNE

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
2.2.0
Keras version: 2.3.0-tf
Transformes 3.4.0


In [4]:
import pandas as pd
dfTrain = pd.read_csv("/content/drive/My Drive/Dataset/penyisihan-datavidia-7-0/train.csv",sep=",")
dfTest = pd.read_csv("/content/drive/My Drive/Dataset/penyisihan-datavidia-7-0/test.csv",sep=",")

In [ ]:
dfTrain.loc[189]

review_id      fab1cc048d2ae4c985f08d0050b6d995
review_text                           PAYAH !! 
category                                      0
Name: 189, dtype: object

In [ ]:
print(f"Row {dfTrain.shape[0]} \n Col {dfTrain.shape[1]}") # row = 14856, col = 3
print(f"Col Names {dfTrain.columns}") 

Row 14856 
 Col 3
Col Names Index(['review_id', 'review_text', 'category'], dtype='object')


In [ ]:
dfTrain = dfTrain.drop(["review_id"] , axis=1) # Dropped
dfTrain.head()

,review_text,category
0,"Penginapannya rapi, bersih, suasana tenang, pe...",1
1,"wifi tidak sampai kamar, tidak tersedia heater...",0
2,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0
3,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0
4,bocor lagi,0


In [ ]:
print(f"How Many Nulls  ? {dfTrain['category'].isnull().sum()}") # 0
print(f"How Many Values For 1 and 0 \n{dfTrain['category'].value_counts()} ") # Imbalance (0 : 12839, 1 : 2017)


How Many Nulls  ? 0
How Many Values For 1 and 0 
0    12839
1     2017
Name: category, dtype: int64 


In [ ]:
def clean_text(text):

  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 
  text_cleaning_num = "(^|\W)\d+"

  
  text = sub(text_cleaning_hash, " ", text).strip()
  text = sub(text_cleaning_num, " ", text).strip()
  text = sub(text_cleaning_re, " ", text).strip()
  text = text.lower().strip()
  text = "".join(text.split("."))  
  out = []
  for word in text.split() :
    out.append(word)

  return pytypo.correct_sentence(" ".join(out).strip())

clean_text("cleaning Gan.. 213213902183 httP:////........ ")

'cleaning gan http'

In [ ]:
def clean_emoji(text):
    emoji_patterns = compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=UNICODE)
    return emoji_patterns.sub(r'', text)
print(f"Before :{dfTrain.loc[14492  ,'review_text']}")
clean_emoji(dfTrain.loc[14492  ,"review_text"])


"""with open('/content/drive/My Drive/Emoji_Dict.p', 'rb') as fuf:
  Emoji_Dict = pickle.load(fuf)

Emoji_Dict = {v : k for k,v in Emoji_Dict.items()}
def clean_emoji(text):
  for emot in Emoji_Dict:
    text = sub(r'('+emot+')'," ".join(Emoji_Dict[emot].replace("," , "").replace(":","").split()),text).strip()

  return text
clean_emoji("I won 🥇 in 🏏 gan")"""

Before :kamarnya sempit, toilet apalagi, water heater y jg ga berfungsi. trus nemuin kelabang di lantai kamar dan di atas kasur, OMG😨😨😨


'with open(\'/content/drive/My Drive/Emoji_Dict.p\', \'rb\') as fuf:\n  Emoji_Dict = pickle.load(fuf)\n\nEmoji_Dict = {v : k for k,v in Emoji_Dict.items()}\ndef clean_emoji(text):\n  for emot in Emoji_Dict:\n    text = sub(r\'(\'+emot+\')\'," ".join(Emoji_Dict[emot].replace("," , "").replace(":","").split()),text).strip()\n\n  return text\nclean_emoji("I won 🥇 in 🏏 gan")'

In [ ]:
dfTrain["review_text"] = dfTrain["review_text"].apply(clean_text)
dfTrain["review_text"] = dfTrain["review_text"].apply(clean_emoji)
# Test Set Should not be Preprocessed :) leave it as is
#dfTest["review_text"] = dfTest["review_text"].apply(clean_emoji)
#dfTest["review_text"] = dfTest["review_text"].apply(clean_text)
print("Cleaned")

Cleaned


In [ ]:

UniqueWord = set()
forCounter = []
def Uniques(df):
  for i in range(len(df)):
    Listeddesu = df.loc[i,"review_text"].split(" ")
    for j in Listeddesu:
      UniqueWord.add(j)
      forCounter.append(j)
Uniques(dfTrain)

In [ ]:
print(f"There are : {len(UniqueWord)} words" )

There are : 13071 words


In [ ]:
ShortWord = []
MediumWord = []

def takeShortestword(word):
  for i in word:
    if len(i) < 5:
      ShortWord.append(i)
  for i in word:
    if len(i) > 5 and len(i) < 8 :
      MediumWord.append(i)
takeShortestword(forCounter)
len(ShortWord) ,len(MediumWord)

(94372, 56903)

In [ ]:
UniqueWord = set()
Uniques(dfTrain)
print(f"There are : {len(UniqueWord)} words" )

There are : 13071 words


In [ ]:
ShortWord = sorted(dict(Counter(ShortWord)).items(),key= lambda dct:dct[1],reverse=True)

In [ ]:
NeedTobefixedWord = []
ConsiderTheWord = []
for i in ShortWord:
  if i[1] > 100:
    NeedTobefixedWord.append(i)
  else:
    ConsiderTheWord.append(i)

len(NeedTobefixedWord),len(ConsiderTheWord)

(140, 2277)

In [ ]:
## Only 200,000 reqs per day / need to be improved using google cloud api
EnglishRow =[91,199,305,342,388,484,490,561,598,878,895,953,1065,1140,1336,1386,1414,1455,1543,1577,1589,1618,1652,1738,1741,1798,1781,1878,
             1904,1919,1969,2144,2273,2349,2476,2533,2714,2754,2794,2814, 2860, 2859, 2929,2970,2988,3070,3167,3259,3281,3289,3291,3402,3504,3535,3549,3567,
             3606,3610,3620,3638,3647,3697,3743,3893,3932,4080,4140,4185,4215,4220,4270,4382,4430,4521,4534,4607,4662,4678,4696,4716,4725,4738,4765,4766,4787,
             4791,4795,4809,4884,4910,4911,4996,5008,5014,5084,5361,5366,5380,5407,5416,5525,5554,5620,5771,5776,5805,5823,5824,5843,5860,5900,5975,6001,6062,6107,6117
             ,6387,6465,6503,6612,6676,6795,6809,6950,7105,7116,7179,7222,7426,7431,7481,7516,7568,7581,7604,7734,7926,8040,8053,8055,8082,8100,8315,8377,8392,8402,8494,
             8601,8653,8691,8719,8865,9032,9039,9129,9142,9240,9260,9421,9458,9486,9497,9474,9586, 9600,9634,9731,9791,9800,9874,9933,10015,10067,10076,10130,10169,10869,10884,10892,10900,10902,10947,11049,11105,11112,11189,11195,11256,11339,11378,11405,11409,
             11719,11815,11842,11867,11905,11920,11980,12054,12060,12067,12102,12134,12191,12301,12311,12323,12370,
             12376,12390,12406,10237,10245,10269,10335,10348,10435,10520,10566,10729,10753,12550,12656 ,12709 ,12719 ,12822,12980,13039,13104 ,13265 ,13342 ,13349 ,13596 ,13672 ,13732 ,13886,13945,13956,14022, 14073, 14189, 14238, 14301, 14490, 14671, 14678, 14682, 14718, 14731, 14740, 14748, 14812, 14822, 14825, 14840]

def TranslateGan(text):
  time.sleep(10)
  translator = Translator()
  ans = translator.translate(text,dest="id")
  print(ans.text)
  return ans.text

for i in EnglishRow:
  dfTrain.loc[i,"review_text"] = TranslateGan(dfTrain.loc[i,"review_text"])

#dfTrain.to_csv("/content/drive/My Drive/EnglishTrain.csv",index=False)

In [ ]:
checktranslated = [91,199,305,342,388,484,490,561,598,878,895,953,1065,1140,1336,1386,1414,1455,1543,1577,1589,1618,1652,1738,1741,1798,1781,1878,
             1904,1919,1969,2144,2273,2349,2476,2533,2714,2754,2794,2814, 2860, 2859, 2929,2970,2988]
for i in checktranslated:
  print(f'{i} -- > {dfTrain.loc[i,"review_text"]} \n {dfTrain.loc[i,"category"]}')

In [60]:
dfTrain=pd.read_csv("/content/drive/My Drive/EnglishTrain.csv")
only1 = dfTrain[dfTrain["category"] == 1]
only0 = dfTrain[dfTrain["category"] == 0]

for i in only1.index:
  print(f"{i} --> {dfTrain.loc[i,'review_text']}")
deleteonly1 = [53,13736 ,2773,4924,10780,13961,6159,10295,9172,4390,13639,10309,12406,3392,28,646,14715 ,14750 ,14559 ,92,161,13033 ,210,13291 ,317,331,335,12493 ,10087,10306,12101,12253,10623,11275 ,385,4174,6853,7309 ,9331 , 7783 ,8870 , 10007 ,477 ,751 ,854,3993  ,857,3734 ,3718,5546   ,911 ,1080,3026,3962   ,1106 ,1155 ,1519 ,1721 ,1768 ,2361,2425 ,3734]
print(len(only0))

0 --> penginapannya rapi bersih suasana tenang pelayannya ramah dan banyak membantu lokasi cukup dekat ke beberapa obyek wisata kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini
7 --> pelayanannya sangat baik kmr yang bersih menu makanannya juga enak to sayang wifinya kutang bagus
12 --> hotel bersih bgt nyaman kolam renang bersih pelayanan hotel baik deket sm indomaret depan hotel ada makanan manado murah dan enak remomended banget pokoknya
13 --> roomnya bersih banget lokasi strategis pusat kota nyaman namun pesan yang double bed dapatnya twin bad gempa lombok bikin takut menginap di hotel bertingkat
28 --> kamar agak lembab lokasi oke
37 --> disini kami sekeluarga sangat diperhatikan dari kami datang sanpai cek out
39 --> papan nama kurang terlihat kurang besar lokasi pinggir jln banyak penjual makan pelayanan kebersihan wifi ok sesuai std airy hanya ac kurang dingin
53 --> kamarnya kurang kedap suara jadi suara dari luar mudah terdengar
54 --> tempat dan kamar bers

In [61]:
for i in only0.index:
  print(f"{i} --> {dfTrain.loc[i,'review_text']}")
delonly0=[5,24,34,46,69,87,165,181,232,234,249,311,316,345,403,415,481,
      500,530,538,594,636,660,684,696,726,728,753,825,837,856,892,906,924,997,1056,1072,1076,1104,1114,1148,
      1158,1177,1261,1271,1275,1425,1573,1614,1641,1794,2027,2093,2337,2366,2406,2416,2799,2812,2891,2444,1859,2466, 2899,3029,3090,3148 ,
      3157,3161,3235,3300,3314,3339,3342,3397,1549,2385,874,1485,3283
      ]


Streaming output truncated to the last 5000 lines.
9065 --> baru sampe hotel dikabarin sama pihak hotel bahwa penuh barulah callcenternya hubungi katanya penuh padahal saya mesinnya dari sore dan baru cek in agak malam ditawarin sih untuk pindah hotel tapi terlanjur kaciwa saya minta refund aja dan beralih ke aplikasi yg lain
9066 --> sehabis mandi air di lantai wc menggenang jd kaya empang kalo malam lantai wc banyak binatang kaya lalat pada mati gitu ac nya jg bocor jadi lantai kamar becek noda di sprei saya maklumi mungkin emg sulit dihilangkan staff nya ramah kok channel tv jg banyak pilihan kasur empuk juga terima kasih ya
9067 --> kamar mandi kotor kumuh kebersihan kurang terjaga
9069 --> kotor tempat nya tidak terawat airy rooms paling jelek deh kecewa
9070 --> malem lampunya matii mulu resepsionistnya udah kaya satpam sikapnya blum waktunya cek out udah di suruh cek out
9071 --> kamar mandi kurang bersih masih bau
9072 --> jangan terlalu kaku dengan wkt cek in jika makar dah si

In [62]:
addtionchangeto1 = [3344,3426,3423,933,1320,3035,4232,3308,4143,3574]
addtionchangeto0 =  [3070,11103,1911,9254,6064,14566,9925,3199,5663,13877,5059,6230,9554,10709,5525,11271,9421,11730
                    ,6426,4678,2792,9722,12455,1475,10348,11120,14323,4256,11192,1583,6897,14073,11799,6358,5138,14269,7763,7457,9876,4740]


In [63]:
desiredIndex1 = [i for i in only1.index if i not in deleteonly1]
desiredIndex0 = [i for i in only0.index  if i not in delonly0]
only1 = only1.loc[desiredIndex1] 
only0 = only0.loc[desiredIndex0]
print(len(only1),len(only0))
for i in addtionchangeto1:
  only0.loc[i,"category"] = 1
for i in addtionchangeto0:
  only1.loc[i,"category"] = 0

1956 12759


In [64]:
only1For1000 = only1[:1900] # only train 1900
onlyZero0 = only0[:5000]

In [72]:
dfTrain=pd.concat([only1For1000,onlyZero0],axis=0)
dfTrain.dropna(inplace=True)
print(len(dfTrain))

6896


In [66]:
dfTrain.index = range(len(dfTrain))
def checkLeakageData():
  """ Check Leak data """
  leakage = False
  Theindex = []
  for key,val in dfTrain["review_text"].items():
    for key2,val2 in dfTest["review_text"].items():
      if(val == val2):
        leakage=True
        Theindex.append((key,val))
  
  print(f"Leakage ? {leakage}")
  return Theindex


In [79]:
def RemoveLeakage():
  """ Digunakan untuk menghapus data yang leak ke test data """
  print(len(dfTrain))
  for i in lol:
      dfTrain.loc[i[0]] = np.nan 
  print(len(dfTrain))
  dfTrain.dropna(inplace=True)
  

In [75]:
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stopwordsObj = set(stopwords.words("indonesian"))
def applyStemmer(text):
  Stemmer = StemmerFactory().create_stemmer()
  return Stemmer.stem(text)
dfTrain["review_text"] = dfTrain["review_text"].apply(lambda sentence: " ".join(word for word in sentence.split(" ") if word not in stopwordsObj))
dfTrain["review_text"] = dfTrain["review_text"].apply(applyStemmer)

In [77]:
dfTrain.to_csv("/content/drive/My Drive/PreprocessedData.csv",index=False)

In [80]:
lol = checkLeakageData()
RemoveLeakage()

Leakage ? False
6896
6896


In [82]:
dfTrain["review_text"].head(10)

0     inap rapi bersih suasana tenang layan ramah ba...
7     layan kmr bersih menu makan enak to sayang wif...
12    hotel bersih bgt nyaman kolam renang bersih la...
13    roomnya bersih banget lokasi strategis pusat k...
37                      keluarga perhati sanpai cek out
39    papan nama lokasi pinggir jln jual makan layan...
54                           kamar bersih harga jangkau
60    suka tinggal twin star hotel solo rumah layan ...
64    lokasi hotel pusat ramai pasar tradisional pan...
71    having great services and strategic locations ...
Name: review_text, dtype: object

In [83]:
from sklearn.feature_extraction.text import CountVectorizer
training = CountVectorizer().fit_transform(dfTrain["review_text"])
print(training.shape)

(6881, 6366)


In [87]:
from sklearn.feature_extraction.text import TfidfTransformer
training_tfidf = TfidfTransformer().fit_transform(training)

In [89]:
training_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(training_tfidf,dfTrain["category"],test_size = 0.10,random_state = 1)

In [127]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]
ThemodelFitted = {}
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(max_depth=45),
    RandomForestClassifier(max_depth=45,n_estimators=250),
    LogisticRegression(),
    SGDClassifier(),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = model
    ThemodelFitted[name] = nltk_model.fit(x_train,y_train)


In [125]:
print("Training Gan")
for name in names:
    predicted = ThemodelFitted[name].predict(x_train)
    print("=========================================")
    print("Using Model {}".format(name))
    print(classification_report(y_train,predicted))
    print("=========================================\n\n")

Training Gan
Using Model K Nearest Neighbors
              precision    recall  f1-score   support

         0.0       0.79      0.99      0.88      4505
         1.0       0.92      0.31      0.46      1687

    accuracy                           0.80      6192
   macro avg       0.86      0.65      0.67      6192
weighted avg       0.83      0.80      0.77      6192



Using Model Decision Tree
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4505
         1.0       1.00      0.99      1.00      1687

    accuracy                           1.00      6192
   macro avg       1.00      1.00      1.00      6192
weighted avg       1.00      1.00      1.00      6192



Using Model Random Forest
              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91      4505
         1.0       1.00      0.50      0.67      1687

    accuracy                           0.86      6192
   macro avg       0.92    

In [128]:
print("Validation Gan")
for name in names:
    predicted = ThemodelFitted[name].predict(x_val)
    print("=========================================")
    print("Using Model {}".format(name))
    print(classification_report(y_val,predicted))
    print("=========================================\n\n")

Validation Gan
Using Model K Nearest Neighbors
              precision    recall  f1-score   support

         0.0       0.78      0.98      0.87       505
         1.0       0.82      0.22      0.35       184

    accuracy                           0.78       689
   macro avg       0.80      0.60      0.61       689
weighted avg       0.79      0.78      0.73       689



Using Model Decision Tree
              precision    recall  f1-score   support

         0.0       0.88      0.87      0.88       505
         1.0       0.66      0.66      0.66       184

    accuracy                           0.82       689
   macro avg       0.77      0.77      0.77       689
weighted avg       0.82      0.82      0.82       689



Using Model Random Forest
              precision    recall  f1-score   support

         0.0       0.86      0.98      0.91       505
         1.0       0.89      0.55      0.68       184

    accuracy                           0.86       689
   macro avg       0.87  

In [ ]:
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=45,n_estimators=250),
    LogisticRegression(),
    SGDClassifier(),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
ThemodelFitted["voting"] = VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1)
ThemodelFitted["voting"].fit(x_train,y_train)

In [131]:
print("Training Gan")
predicted = ThemodelFitted["voting"].predict(x_train)
print("=========================================")
print("Using Model voting")
print(classification_report(y_train,predicted))
print("=========================================\n\n")

print("Validation Gan")
predicted = ThemodelFitted["voting"].predict(x_val)
print("=========================================")
print("Using Model voting")
print(classification_report(y_val,predicted))
print("=========================================\n\n")

Training Gan
Using Model voting
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      4505
         1.0       0.97      0.83      0.89      1687

    accuracy                           0.95      6192
   macro avg       0.95      0.91      0.93      6192
weighted avg       0.95      0.95      0.94      6192



Validation Gan
Using Model voting
              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93       505
         1.0       0.89      0.68      0.77       184

    accuracy                           0.89       689
   macro avg       0.89      0.83      0.85       689
weighted avg       0.89      0.89      0.89       689





In [132]:
## Build the Deep Learning model
from transformers import AutoTokenizer,TFAutoModel,AutoModel
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
IndoBert = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p1")

Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [133]:
from sklearn.model_selection import train_test_split

In [136]:
X_train,x_val,y_train,y_val = train_test_split(dfTrain["review_text"],dfTrain["category"],test_size=0.10,random_state=1)
len(X_train), len(y_train)
print(y_val.value_counts())
cp_ytrain = y_train.copy()

0.0    505
1.0    184
Name: category, dtype: int64


In [137]:
tokens_dfTrain = {
    "review_text": tokenizer.batch_encode_plus(
        X_train.tolist(),
        max_length = MAX_LENGTH,
        pad_to_max_length = True,
        truncation = True)
}
tokens_dfVal = {
    "review_text": tokenizer.batch_encode_plus(
        x_val.tolist(),
        max_length= MAX_LENGTH,
        truncation=True,
        pad_to_max_length= True
    )

    }
train_data ={
        "review_text": tf.convert_to_tensor(tokens_dfTrain["review_text"]["input_ids"]),
        "review_text_mask":tf.convert_to_tensor(tokens_dfTrain["review_text"]["attention_mask"])
}
val_data = {
        "review_text" : tf.convert_to_tensor(tokens_dfVal["review_text"]["input_ids"]),
        "review_text_mask" : tf.convert_to_tensor(tokens_dfVal["review_text"]["attention_mask"])
}
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)



y_train = tf.keras.utils.to_categorical(y_train,2)
y_val = tf.keras.utils.to_categorical(y_val,2)
print(y_train[1:5])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[[1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [138]:
dataset = (tf.data.Dataset.from_tensor_slices((train_data,y_train))
.batch(32)
.prefetch(AUTO))

dataVal = (tf.data.Dataset.from_tensor_slices((val_data,y_val))
.batch(32)
.prefetch(AUTO)
.cache())

In [144]:
def CreateDLModel():

  ids1 = tf.keras.layers.Input(shape=(MAX_LENGTH,),dtype=tf.int32,name="review_text")
  att1 = tf.keras.layers.Input(shape=(MAX_LENGTH,),dtype=tf.int32,name="review_text_mask")


  embedding,SeqOutput = IndoBert(ids1,attention_mask=att1)

  finlayer = tf.keras.layers.Dropout(0.4)(embedding)
  finlayer = tf.keras.layers.Dense(128,tf.nn.relu)(finlayer)
  
  finlayer = tf.keras.layers.Flatten()(finlayer)
  theFinal = tf.keras.layers.Dropout(0.55)(finlayer)
  theFinal = tf.keras.layers.Dense(2,activation=tf.nn.softmax)(theFinal)


  final = tf.keras.Model([ids1,att1],[theFinal])
  METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
  ] 
  
  optimizer = tf.keras.optimizers.Adam()
  final.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(),optimizer=optimizer,
               metrics=METRICS)

  for layer in final.layers:
    layer.trainable = False
  for layer in final.layers[-5:]:
      layer.trainable = True

  return final

models = CreateDLModel()

In [145]:
import math
def create_learning_rate_scheduler(max_learn_rate=1e-2,
                                   end_learn_rate=1e-6,
                                   warmup_epoch_count=3,
                                   total_epoch_count=15):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = max_learn_rate
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [146]:
tf.keras.utils.plot_model(model,"modelarhcitect.png",show_shapes=True)

AttributeError: ignored

In [147]:
models.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
review_text (InputLayer)        [(None, 45)]         0                                            
__________________________________________________________________________________________________
review_text_mask (InputLayer)   [(None, 45)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 45, 768), (N 124441344   review_text[0][0]                
__________________________________________________________________________________________________
dropout_39 (Dropout)            (None, 45, 768)      0           tf_bert_model[1][0]              
____________________________________________________________________________________________

In [142]:
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience = 10)

In [ ]:
history3=models.fit(x = dataset,epochs = 20,validation_data=dataVal,callbacks = [EarlyStop, create_learning_rate_scheduler(max_learn_rate=1e-4,
                                                    end_learn_rate=1e-6,
                                                    warmup_epoch_count=3,
                                                    total_epoch_count=15),EarlyStop])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/20
194/194 [==============================] - 26s 136ms/step - loss: 0.2612 - tp: 5378.0000 - fp: 814.0000 - tn: 5378.0000 - fn: 814.0000 - accuracy: 0.8685 - precision: 0.8685 - recall: 0.8685 - auc: 0.9402 - val_loss: 0.2694 - val_tp: 607.0000 - val_fp: 82.0000 - val_tn: 607.0000 - val_fn: 82.0000 - val_accuracy: 0.8810 - val_precision: 0.8810 - val_recall: 0.8810 - val_auc: 0.9441 - lr: 1.0000e-04

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 2/20
194/194 [==============================] - 24s 122ms/step - loss: 0.1291 - tp: 5597.0000 - fp: 595.0000 - tn: 5597.0000 - fn: 595.0000 - accuracy: 0.9039 - precision: 0.9039 - recall: 0.9039 - auc: 0.9663 - val_loss: 0.1199 - val_tp: 612.0000 - val_fp: 77.0000 - val_tn: 612.0000 - val_fn: 77.0000 - val_accuracy: 0.8882 - val_precision: 0.8882 - val_recall: 0.8882 - val_auc: 0.9484 - lr: 1.0000e-04

Epoch 00003: LearningRateScheduler 

In [ ]:
from sklearn.metrics import classification_report
y_valPred = np.argmax(model.predict(dataset),axis = 1)


In [ ]:
print(classification_report(np.argmax(y_train,axis = 1),y_valPred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4925
           1       0.96      0.95      0.96      1887

    accuracy                           0.98      6812
   macro avg       0.97      0.97      0.97      6812
weighted avg       0.98      0.98      0.98      6812



In [ ]:
OnlyZero=pd.read_csv("/content/drive/My Drive/EnglishTrain.csv")
OnlyZer= OnlyZero[OnlyZero["category"] == 0][7000:]
OnlyOne = OnlyZero[OnlyZero["category"]==1][1900:]
OnlyZero=pd.concat([OnlyOne,OnlyZer],axis=0)
OnlyZero.dropna(inplace=True)



x_train,x_val,y_train,y_val = train_test_split(OnlyZero["review_text"],OnlyZero["category"],test_size=0.000001)

tokens_dfTrain = {
    "review_text": tokenizer.batch_encode_plus(
        x_train.tolist(),
        max_length = MAX_LENGTH,
        pad_to_max_length = True,
        truncation = True)
}


train_data ={
        "review_text": tf.convert_to_tensor(tokens_dfTrain["review_text"]["input_ids"]),
        "review_text_mask":tf.convert_to_tensor(tokens_dfTrain["review_text"]["attention_mask"])
}

y_train = tf.convert_to_tensor(y_train)

y_train = tf.keras.utils.to_categorical(y_train,2)
print(y_train[1:5])
dataset = (tf.data.Dataset.from_tensor_slices((train_data,y_train))
.batch(32)
.prefetch(AUTO))

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
y_valPred = np.argmax(model.predict(dataset),axis = 1)
print(classification_report(np.argmax(y_train,axis = 1),y_valPred))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      5823
           1       0.31      0.94      0.47       117

    accuracy                           0.96      5940
   macro avg       0.65      0.95      0.72      5940
weighted avg       0.99      0.96      0.97      5940



In [ ]:

dfcheck=pd.DataFrame(x_train)
dfcheck["category"] = OnlyZero["category"]
dfcheck["pred"] = y_valPred
lol = dfcheck[dfcheck["category"] != dfcheck["pred"]].index
for i in lol:
  print(f'{i} --> {dfcheck.loc[i,"review_text"]}\n True : {dfcheck.loc[i,"category"]} \n Predicted:{dfcheck.loc[i,"pred"]} \n')
dele =[]

14288 --> tolong segera perbaiki lokasi hotel pada airy maps kami kesulitan menemukan hotel ini
 True : 0 
 Predicted:1 

13587 --> pelayanan yang sangat ramah dan baik hati
 True : 0 
 Predicted:1 

10356 --> hotel biasa saja air hangat ok wifi kurang sarapan lumayan kwcewa dengan snack yang disediain airy sdh expired
 True : 0 
 Predicted:1 

10718 --> kamar bersih sesuai dgn fotonya cek in mudah ini kali kedua saya menginap pengalaman sebelumnya baik hanya saja kali ini saya agak kesusahan menghubungi staf
 True : 0 
 Predicted:1 

13108 --> kamarnya oke cuma airnya kecil
 True : 0 
 Predicted:1 

9112 --> harga sesuai dengan fasilitas
 True : 0 
 Predicted:1 

10365 --> kamar lumayan cuman agak berisik karena dekat pub dan karaoke makanannya biasa aja dan kurang variasi
 True : 0 
 Predicted:1 

10773 --> dapat kamar bawah tanah wifi susah dapat sinyal sinyal hp juga susah sekali air hangat sangat kecil alirannya tetapi harga kamar sangat murah terutama dibanding hotel lain pas mom

In [ ]:
#model.save_weights("/content/drive/My Drive/BestModel/")

In [ ]:
model.load_weights("/content/drive/My Drive/BestModel/")

In [ ]:
# Test ---
#dfTest["review_text"] = dfTest["review_text"].apply(clean_text)
#dfTest["review_text"] = dfTest["review_text"].apply(clean_emoji)
#dfTest["review_text"] = dfTest["review_text"].apply(FixIt)

tokens_dftest = {
    "review_text": tokenizer.batch_encode_plus(
        dfTest["review_text"].tolist(),
        max_length= MAX_LENGTH,
        truncation=True,
        pad_to_max_length= True
    )}

dfTestdataset = {
        "review_text": tf.convert_to_tensor(tokens_dftest["review_text"]["input_ids"]),
        "review_text_mask":tf.convert_to_tensor(tokens_dftest["review_text"]["attention_mask"])
}


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
y_pred = np.argmax(model.predict(dfTestdataset),axis = 1)
y_pred[1:15]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
dfTestAns = pd.DataFrame(dfTest["review_id"])
dfTestAns['category'] =y_pred

In [ ]:
dfTestAns.to_csv("SubmitGan.csv",index=False)

In [ ]:
y_pred2 = np.argmax(model.predict(dfTestdataset),axis = 1)
dfTestAns2 = pd.DataFrame(dfTest["review_id"])
dfTestAns2['category'] =y_pred2

In [ ]:
print(dfTestAns["category"].value_counts() )
print(dfTestAns2['category'].value_counts())

0    3275
1     439
Name: category, dtype: int64
0    3277
1     437
Name: category, dtype: int64


In [ ]:
dfTestAns2["category"].value_counts()


0    3187
1     527
Name: category, dtype: int64

In [ ]:
dfTestAns2.to_csv("/content/drive/My Drive/ans.csv",index=False)

In [ ]:
dfTestAns3 = dfTest.copy()
dfTestAns3["yo"] = y_pred2
dfTest1Only=dfTestAns3[dfTestAns3["yo"] == 1]
for i in dfTest1Only.index:
  print(f'{i} ---> {dfTest1Only.loc[i,"review_text"]} pred: {dfTest1Only.loc[i,"yo"]}\n')
tandai=[37,64,63,107,146,158,177,]

4 ---> Lumayan untuk guest house dengan standart harga segitu. pred: 1

11 ---> So far so good. Bersih.. Nyaman.. AC juga dingin. Aman. Tenang. Cuma kadang toilet nya mampet. pred: 1

16 ---> Hotel, lokasi dan harga yg sangat nyaman.  pred: 1

20 ---> Air untuk mandi rasanya asin, sikat gigi harus pakai aqua botol. Lokasi sulit dijangkau, tidak ada tanda Airy seperti kebanyakan tempat lain sehingga harus tanya sana sini. Airy ini murah dan memang murahan. Tidak recommended laah! pred: 1

23 ---> Comfortable, clean and tidy Cheap, and recommend pred: 1

26 ---> Kamar yang luas, dekat dengan pantai kuta, sebelah ada warung makanan murah dan halal. Minus tidak a pred: 1

30 ---> Nyaman dan bagus lokasinya mudah di cari. pred: 1

62 ---> Pelayanan memuaskan, staf ramah, mantap! pred: 1

63 ---> Wifi nya tidak sampai kelantai 2, dikamar mandi , tidak ada cantelan nya pred: 1

66 ---> I think this hotel is very recommended for traveler. The facilities is good enough related the cost. pred: 1

In [ ]:
dfTestAns3 = dfTest.copy()
dfTestAns3["yo"] = y_pred2
dfTest1Only=dfTestAns3[dfTestAns3["yo"] == 0]
for i in dfTest1Only.index:
  print(f'{i} ---> {dfTest1Only.loc[i,"review_text"]} pred: {dfTest1Only.loc[i,"yo"]}\n')
tandai=[37,64,63,107,146,158,177,]

Streaming output truncated to the last 5000 lines.
1207 ---> kelebihan: kamar bersih & wifi lumayan cpt

kekurangan: kamar sempit, berisik, banyak nyamuk, tidak ada cahaya matahari  pred: 0

1208 ---> - jaringan wi-fi super lelet dan susah login 
- sarapan ala kadarnya  pred: 0

1209 ---> ktp saya tertinggal...ga dibalikin saat check out dan ga ada pemberitahuan pdhal sudah meninggalkan no hp.
gatau mau kontak kemana..

mohon dibantu pred: 0

1210 ---> Keterangan tentang persyaratan menginap tidak lengkap, pelayanan tidak baik, mengecewakan. pred: 0

1212 ---> Penjaganya ramah, sayang kamar mandinya bau apek sampai ke dalam kamar meskipun sudah ada blower pred: 0

1213 ---> kamar bersih dan peralatan tersedia. pelayanan juga lumayan baik dan cepat. tempat tidur brrsih. shower tdk memadai di lantai 4 tempat saya berada air mya kecil dan tdk memadai utk mandi saat pagi  jam 5.  dan saluran air menyeluarkan bau tidk sedap di pagi hari... pred: 0

1215 ---> AC tidak dingin Suite 29 pred: 0

In [ ]:
dfTestAns2.to_csv("ans2.csv",index=False)

In [ ]:
for i in range(len(dfTest["review_text"])):
  print(f"loc {i} --> {dfTest.loc[i,'review_text']}")

                 #3080 < DATA PARAH >
dfTestEnglish.to_csv("TestArk.csv")

NameError: ignored